In [1]:
from typing import Dict
import numpy as np
import ray

In [ ]:
ds = ray.data.read_tfrecords(["/mnt/750G/data/droid/1.0.0/"])

In [ ]:
ds

In [ ]:
data = ds.take(17)

In [4]:
import io
from PIL import Image

In [6]:
for i in range(len(data)):
    for u in range(len(data[i]['steps/observation/exterior_image_1_left'])):
        img = Image.open(io.BytesIO(data[i]['steps/observation/exterior_image_1_left'][u]))
        tmp = img.save(f"/mnt/750G/data/droid_right/{i}_{u}.png")

In [49]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import io

In [ ]:
i[0]

In [ ]:
plt.imshow(Image.open(io.BytesIO(i[1])))

In [2]:
ds = ray.data.read_images("/mnt/750G/data/droid_all")

2024-12-14 16:52:16,008	INFO worker.py:1821 -- Started a local Ray instance.


In [3]:
ds

2024-12-14 16:52:40,655	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-14_16-52-13_202376_372677/logs/ray-data
2024-12-14 16:52:40,655	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


Running 0: 0.00 row [00:00, ? row/s]

- ReadImage 1: 0.00 row [00:00, ? row/s]

2024-12-14 16:52:51,741	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-14_16-52-13_202376_372677/logs/ray-data
2024-12-14 16:52:51,743	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage]


Running 0: 0.00 row [00:00, ? row/s]

- ReadImage 1: 0.00 row [00:00, ? row/s]

Dataset(
   num_rows=22056,
   schema={image: numpy.ndarray(shape=(180, 320, 3), dtype=uint8)}
)